<a href="https://colab.research.google.com/github/mirjanaa/Bee-Colony-Optimization/blob/main/money.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import random

In [69]:
class FoodSource:

  def __init__(self, bounds, obj_function, target):
    self.din_1 = random.randint(0, bounds[0])
    self.din_2 = random.randint(0, bounds[1])
    self.din_5 = random.randint(0, bounds[2])
    self.din_10 = random.randint(0, bounds[3])
    self.din_20 = random.randint(0, bounds[4])

    self.position = np.array([self.din_1, self.din_2, self.din_5, self.din_10, self.din_20]) #necemo ovako? Brisi ako ne treba
    self.target = target
    self.obj_value = obj_function(self.din_1, self.din_2, self.din_5, self.din_10, self.din_20, self.target)
    self.fitness = self.calc_fitness()
    self.trial = 0
    self.probability = 0

  def __lt__(self, other):
    return self.obj_value < other.obj_value

  def __gt__(self, other):
    return self.fitness > other.fitness


  def calc_fitness(self):
    if self.obj_value >= 0:
      return 1 / (1 + self.obj_value)
    return 1 + abs(self.obj_value)
    




In [70]:
def obj_function(din_1, din_2, din_5, din_10, din_20, target):
  alpha = 1 #parametar tezine vrednosti blizine resenja
  beta = 1  #parametar tezine broja novcica

  value = 1*din_1 + 2*din_2 + 5*din_5 + 10*din_10 + 20*din_20
  count = din_1 + din_2 + din_5 + din_10 + din_20

  return alpha*abs(target - value) + beta*(count)


In [71]:
bounds = np.array([5, 10, 5, 15, 2])


In [72]:
def generate_new_solution():
  pass

In [73]:
POPULATION_SIZE = 50
NUM_OF_ITERATIONS = 100
LIMIT = 5
target = 100


population = [FoodSource(bounds, obj_function, target) for _ in range(POPULATION_SIZE)]

best_solution = min(population)

for k in range(NUM_OF_ITERATIONS):
  #### Employed Bee Phase ####
  for i in range(POPULATION_SIZE):
    generate_new_solution()

  #### Onlooker Bee Phase ####
  max_fitness = max([population[i].fitness for i in range(POPULATION_SIZE)])
  for i in range(POPULATION_SIZE):
    population[i].probability = 0.9 * (population[i].fitness / max_fitness) + 0.1


  m = 0
  n = 0

  while m < POPULATION_SIZE and n < POPULATION_SIZE:
    rand_value = random.uniform(0, 1)
    if rand_value < population[n].probability:
      generate_new_solution()
      m += 1
    n = (n % POPULATION_SIZE) + 1

  best_solution = min(min(population), best_solution)

  #### Scout Bee Phase ####
  max_index = -1
  max_trial = float('-inf')
  for index, p in enumerate(population):
    if p.trial > max_trial:
      max_trial = p.trial
      max_index = index


  if population[max_index].trial > LIMIT:
    population[max_index] = FoodSource(bounds, obj_function, target)

  best_solution = min(min(population), best_solution)


print("position: ", [i for i in best_solution.position])
print("objective value: ", best_solution.obj_value)
print("fitness value: ", best_solution.fitness)

position:  [1, 4, 3, 7, 0]
objective value:  21
fitness value:  0.045454545454545456
